In [6]:
#https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia
#image preprocessing
import cv2
import numpy as np
import os
from PIL import Image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [8]:
# window to viewport transformation
def func(folder_path,folder_path1):
    for img in os.listdir(folder_path):
        copy=img
        path1=(folder_path+'/'+img)
        img=load_img(path=path1)
        img=img_to_array(img)
        first_dimension,second_dimension,third_dimension=img.shape
        
        xmin=0
        ymin=0
        xmax=first_dimension
        ymax=second_dimension
        
        umin=0
        vmin=0
        umax=32
        vmax=32
        
        sx=(umax-umin)/(xmax-xmin)
        sy=(vmax-vmin)/(ymax-ymin)
        
        newimage=np.zeros(shape=(32,32,3),dtype=int)
        
        #for copying the image matrix in new image matrix
        for i in range(0,32):
            i1=((i-umin)/sx)+xmin
            for j in range(0,32):
                j1=((j-vmin)/sy)+ymin
                i1=int(i1)
                j1=int(j1)
                newimage[i][j]=img[i1][j1]
        path=(folder_path1+'/')
        cv2.imwrite(os.path.join(path , copy), newimage)
        cv2.waitKey(0)
folder_path='chestxray//train//NORMAL'
folder_path1='compressed32//train//NORMAL'
func(folder_path,folder_path1)

folder_path='chestxray//train//PNEUMONIA'
folder_path1='compressed32//train//PNEUMONIA'
func(folder_path,folder_path1)


folder_path='chestxray//test//NORMAL'
folder_path1='compressed32//test//NORMAL'
func(folder_path,folder_path1)

folder_path='chestxray//test//PNEUMONIA'
folder_path1='compressed32//test//PNEUMONIA'
func(folder_path,folder_path1)      

In [9]:
#CNN architecture

from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [12]:
#initialize CNN
classifier=Sequential()
#feature detector or convolution layer
classifier.add(Convolution2D(32,(3,3),input_shape=(32,32,3),activation='relu'))
#maxpooling
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [13]:
#feature detector and MAXpooling
classifier.add(Convolution2D(32,(3,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))


In [14]:
#full connection
classifier.add(Flatten())
classifier.add(Dense(output_dim=128,activation='relu'))
classifier.add(Dense(output_dim=128,activation='relu'))
classifier.add(Dense(output_dim=1,activation='sigmoid'))
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

C:\Users\ssmma\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ssmma\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  after removing the cwd from sys.path.
C:\Users\ssmma\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  """


In [15]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
training_set = train_datagen.flow_from_directory('compressed32//train',
                                                target_size=(32, 32),
                                                batch_size=32,
                                                class_mode='binary')
                                                

                                                

Found 5216 images belonging to 2 classes.


In [19]:

test_set = test_datagen.flow_from_directory('compressed32//test',
                                             target_size=(32,32),
                                             batch_size=32,
                                             class_mode='binary')

Found 624 images belonging to 2 classes.


In [20]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=5216,
        epochs=5,
        validation_data=test_set,
        validation_steps=624)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
5216/5216 [==============================] - 490s 94ms/step - loss: 0.1491 - acc: 0.9401 - val_loss: 0.3171 - val_acc: 0.8955
Epoch 2/5
5216/5216 [==============================] - 520s 100ms/step - loss: 0.0866 - acc: 0.9671 - val_loss: 0.3248 - val_acc: 0.8850
Epoch 3/5
5216/5216 [==============================] - 514s 99ms/step - loss: 0.0636 - acc: 0.9760 - val_loss: 0.3064 - val_acc: 0.8991
Epoch 4/5
5216/5216 [==============================] - 517s 99ms/step - loss: 0.0501 - acc: 0.9810 - val_loss: 0.6551 - val_acc: 0.8399
Epoch 5/5
5216/5216 [==============================] - 484s 93ms/step - loss: 0.0408 - acc: 0.9849 - val_loss: 0.3813 - val_acc: 0.9089


In [21]:

from keras.preprocessing.image import load_img
falsepositive=0
falsenegative=0
truepositive=0
truenegative=0

In [22]:
folder_path='compressed32//test//NORMAL'
for img in os.listdir(folder_path):
        copy=img
        path1=(folder_path+'/'+img)
        img=load_img(path=path1)
        img=img_to_array(img)
        img=img.reshape((1,img.shape[0],img.shape[1],img.shape[2]))
        output=classifier.predict(img)
        if(output[0][0]==1):
            falsepositive=falsepositive+1
        else:
            truenegative=truenegative+1

In [23]:
folder_path='compressed32//test//PNEUMONIA'
for img in os.listdir(folder_path):
        copy=img
        path1=(folder_path+'/'+img)
        img=load_img(path=path1)
        img=img_to_array(img)
        img=img.reshape((1,img.shape[0],img.shape[1],img.shape[2]))
        output=classifier.predict(img)
        if(output[0][0]==1):
            truepositive=truepositive+1
        else:
            falsenegative=falsenegative+1

In [24]:
precision=truepositive/(truepositive+falsepositive)
recall=truepositive/(truepositive+falsenegative)
accuracy=(truepositive+truenegative)/(truepositive+truenegative+falsepositive+falsenegative)
f1score=2*(precision*recall)/(precision+recall)



In [ ]:
classifier.save_weights('savedweights5.h5')
classifier.load_weights('savedweights5.h5')